In [1]:
#we will be making a review summarizer using the langchain library.
# get revews of a business or a location from mongodb and summarize them using langchain

import pymongo
import json
from bson import ObjectId
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate

load_dotenv(dotenv_path="../config.env")

True

In [6]:
# setup mongodb connection
def connect_to_db():
    db_client = pymongo.MongoClient(os.getenv("MONGODB_URI_LOCAL"))
    db = db_client['OdysseumDataBase']
    print(db.list_collection_names())
    return db

def get_reviews_by_business(business_id):
    db = connect_to_db()
    collection = db['Review']
    reviews = collection.find({"entityId": ObjectId(business_id)})
    return reviews

#run test on the get_reviews_by_business function
reviews = list(get_reviews_by_business("6781353badab4c338ff55148"))

for review in reviews:
    print(review)


['Service', 'PostReport', 'Report', 'Review', 'Business', 'Chat', 'Booking', 'Comment', 'Message', 'Location', 'Post', 'User', 'Itinerary']
{'_id': ObjectId('67813933b4e348d916a9047d'), 'creatorId': ObjectId('672f358fb3e56fac046d76a5'), 'entityType': 'Location', 'entityId': ObjectId('6781353badab4c338ff55148'), 'rating': 4.5, 'reviewContent': 'This is some test review content.', 'imageUrls': [], 'upvotes': [], 'downvotes': [], 'createdAt': datetime.datetime(2025, 1, 10, 15, 13, 55, 665000), 'updatedAt': datetime.datetime(2025, 1, 10, 15, 13, 55, 665000), '__v': 0}
{'_id': ObjectId('6784c5849514a22d7407e4f9'), 'creatorId': ObjectId('672f358fb3e56fac046d76a5'), 'entityType': 'Location', 'entityId': ObjectId('6781353badab4c338ff55148'), 'rating': 1.5, 'title': 'Hello review', 'reviewContent': 'Another test review and it works!! Without photos for now', 'imageUrls': [], 'upvotes': [], 'downvotes': [], 'createdAt': datetime.datetime(2025, 1, 13, 7, 49, 24, 24000), 'updatedAt': datetime.date

In [ ]:
#langchain setup

def get_langchain_api_key():
    return os.getenv("GROQ_API_KEY")

def initialize_llm_client():
    # api_key = get_langchain_api_key()
    # chat_groq = ChatGroq(api_key)
    model_name="llama-3.1-8b-instant"
    llm_client = ChatGroq(model=model_name)
    return llm_client

def summarize_reviews(reviews):
    llm_client = initialize_llm_client()
    
    review_texts = []
    for review in reviews:
        # Format each review with relevant information
        review_text = f"Rating: {review.get('rating', 'N/A')}/5, Title: {review.get('title', 'N/A')}, Content: {review.get('reviewContent', 'N/A')}"
        # print(review_text)
        review_texts.append(review_text)
    
    # Join all review texts
    all_reviews = "\n\n".join(review_texts)

    # print(all_reviews)
    
    # # Create a proper prompt template with input variables
    prompt_template = PromptTemplate(
        input_variables=["reviews"],
        template="""
        You are an expert summariser of user reviews for businesses and tourist destinations.
        You are tasked with summarising the reviews of a business or tourist destination. The summary should be short and concise, 
        and should capture the essence of the reviews. It should also provide an overview of user sentiments towards the business or destination.
        The reviews are written by users who have visited the business or destination. If you find some user with an interesting opinion, be sure to include it in the summary.
        
        Input:
        List of reviews related to the business or destination: {reviews}

        Output:
        A maximum of 6-7 lines paragraph summarising the reviews and user sentiments towards the business or destination.

        DO NOT add any extra information. Only the summary is required.
        """
    )
    
    # Invoke the LLM with the reviews
    chain = prompt_template | llm_client
    summary = chain.invoke({"reviews": all_reviews})
    
    return summary

#run test on the summarize_reviews function
summary = summarize_reviews(reviews)
# print(summary)

In [11]:
print(summary.content)

Based on the provided reviews, the overall sentiment towards the business or destination is mixed. Most users seem to appreciate the experience, as seen in the 4.5-star review with positive content. However, some users have had disappointing experiences, like the 1.5-star review mentioning it works fine without photos. Another user had an average experience, stating it was a test review. The reviews suggest a need for improvement in certain areas to meet user expectations.
